# License information
Demo by Wouter G. Ellenbroek.

Distributed under the EUPL-1.0 license.

See https://github.com/wouterel/lammps-polymer-workshop for details.

# Installing LAMMPS and perconet into the Runtime
This only needs to be done once with each Colab Runtime. It is unclear how long Runtimes last. They  survive short absences like browser restarts, but will certainly not last more than a few hours when idle.

NOTE: During the execution of this section, which may take a few minutes, your session will "crash" and restart. This is expected behavior.

This section can be collapsed when installing is finished.

This section can be skipped if you downloaded this notebook and are using it locally in an environment in which lammps and perconet are already installed.

In [ ]:
try:
    import perconet
    print(f"perconet {perconet.__version__} is already installed.")
except ModuleNotFoundError:
    !pip install perconet
try:
    import lammps
    print(f"lammps {lammps.__version__} is already installed.")
except ModuleNotFoundError:
    !pip install lammps==2024.8.29.1.0

# Test if LAMMPS is installed

In [ ]:
try:
    import lammps
    lmp = lammps.lammps()
    lmp.close()
    print("LAMMPS OK")
except:
    print("LAMMPS not correctly installed")


# Module imports, settings, and function definitions

Run this section every time the runtime restarts.


In [ ]:
import os
import shutil
import lammps
import numpy as np
import perconet

config = {
    "home": os.getcwd(),
    "scripts_folder": "scripts",
    "script_filename": "tetraPEG_gelation.in"
}

try:
    import google.colab
    config["colab"] = True
    print("Running in Google Colab")
except ModuleNotFoundError as e:
    config["colab"] = False
    print("Running outside of Google Colab")

In [ ]:
if not os.path.exists(config["scripts_folder"]):
    print(f"Creating scripts folder {config['scripts_folder']}")
    os.mkdir(config["scripts_folder"])
else:
    if not os.path.isdir(config["scripts_folder"]):
        print(f"File {config['scripts_folder']} already exists but it is not a directory.")
        print("Please choose a different name for your folder")
    else:
        print("Using existing folder for scripts")

In [ ]:
def create_lammps_dict(concentration, Nstars, seed):
    lammps_dict = {
        "molA": "../scripts/tetraPEG10_A.dat",
        "molB": "../scripts/tetraPEG10_B.dat",
        "conc_val": str(concentration),
        "Nstars": str(Nstars),
        "seed": str(seed),
        "fileoutput": f"N{Nstars}_conc{concentration}_seed{seed}"}
    return lammps_dict

In [ ]:
def create_output_folder(fileoutput):
    output_folder = f"out_{fileoutput}"
    output_fullpath =  os.path.join(config["home"], output_folder)
    try:
        os.mkdir(output_folder)
        print(f"Created output folder {output_folder}")
    except FileExistsError:
        print("WARNING: Output folder already exists.")
        print(f"If {output_folder} contains data it could be overwritten.")
    finally:
        return output_folder, output_fullpath

In [ ]:
def zip_results(output):
    shutil.make_archive(output, 'zip', output)
    if config["colab"]:
        google.colab.files.download(f"{output}.zip")

# Generating molecule files

In [ ]:
os.chdir(config["home"])

molecule_A = {"bead label": "A",
              "bead type": 2,
              "arm length": 10}
molecule_B = {"bead label": "B",
              "bead type": 3,
              "arm length": 10}

import numpy as np
def write_tetrapeg_molecule_file(molecule_parameters):
    type_endbead = molecule_parameters["bead type"]
    arm_length = molecule_parameters["arm length"]
    bead_label = molecule_parameters["bead label"]
    filename = f"tetraPEG{arm_length}_{molecule_parameters['bead label']}.dat"
    full_filename = os.path.join(config["scripts_folder"], filename)
    print(f"Generating file for molecule {bead_label} with arm length {arm_length} and endbead type {type_endbead}")
    n_bonds = 4 * arm_length
    n_atoms = n_bonds + 1
    pos = np.zeros((n_atoms, 3), dtype=float)
    beadtype = np.full((n_atoms), 6)
    beadtype[0] = 1  # center bead
    bonds = np.zeros((n_bonds, 2), dtype=int)
    bonds[:, 0] = range(1, n_bonds+1)
    bonds[:, 1] = range(2, n_bonds+2)
    dist = np.asarray(range(1,arm_length+1))
    for arm in range(4):
        direction = arm % 2  # this maps to x, y, x, y
        sign = 1 - arm + direction  # this maps to 1, 1, -1, -1
        pos[arm * arm_length + 1: (arm+1) * arm_length + 1, direction] = sign * dist
        beadtype[(arm+1) * arm_length] = type_endbead
        bonds[(arm) * arm_length, 0] = 1

    with open(full_filename, 'w') as f:
        print(f"  Writing {full_filename}")
        f.write(f"\n\n{n_atoms} atoms\n{n_bonds} bonds\n\n")
        f.write("Coords\n")
        for i, pos_i in enumerate(pos):
            f.write(f"\n{(i+1):5}")
            for ax in range(3):
                f.write(f" {pos_i[ax]:8.1f}")
        f.write("\n\nTypes\n\n")
        for i, type_i in enumerate(beadtype):
            f.write(f"{(i+1):5} {type_i:5}\n")
        f.write("\nBonds\n\n")
        for i, bond_i in enumerate(bonds):
            f.write(f"{(i+1):5}     1 {bond_i[0]:6} {bond_i[1]:6}\n")

write_tetrapeg_molecule_file(molecule_A)
write_tetrapeg_molecule_file(molecule_B)

# Generating LAMMPS script
To avoid losing your edits, running this block will generate the script **only if there is currently no script with the same filename**. Delete the current script from the `scripts` folder (or rename it) if you want to re-generate the default script.

In [ ]:
full_filename = os.path.join(config["scripts_folder"], config["script_filename"])
print(full_filename)
if os.path.exists(full_filename):
    print(f"Not generating script because {full_filename} already exists")
else:
    with open(full_filename, "w") as f:
        f.write("""log log_${fileoutput}.txt

# basic simulation settings
units lj
atom_style bond

# define simulation box
variable init_size equal "800"
region mybox block 0 ${init_size} 0 ${init_size} 0 ${init_size}

# create box with extra parameters to reserve space for adding molecules
create_box 6 mybox bond/types 2 extra/special/per/atom 100 extra/bond/per/atom 4

# specify files that contain molecule data
molecule stara ${molA}
molecule starb ${molB}

# Set masses of atom types
# Types 4 and 5 are unused here but need to be set anyway
mass 1 1
mass 2 1
mass 3 1
mass 4 1
mass 5 1
mass 6 1

# create_atoms <offset> random <number to make> <random seed> <region> mol <mol id> <another random seed>
create_atoms 0 random ${Nstars} 1 mybox mol stara 1337
create_atoms 0 random ${Nstars} 2 mybox mol starb 1338

# define some variables
variable   conc equal ${conc_val}
variable   r0 equal "1"
variable   sigma_coeff equal "1.3"
variable   sigma equal "v_r0 * v_sigma_coeff"
variable   rcut equal "v_sigma * 2.0 ^ (1.0/6.0)"  # Here F = 0 for lj/cut

variable   v_init equal "v_init_size*v_init_size*v_init_size"
variable   v_final equal "atoms * ((v_sigma)^3)/(v_conc/100)"
variable   boxsize equal "v_v_final^(1.0/3.0)"
variable   scale equal "(v_v_final/v_v_init)^(1.0/3.0)"
variable   compression_steps equal "ceil(100000/(v_conc))"
variable   n_mon equal "v_Nstars * 2"

print "Number of steps for compression: ${compression_steps}"
print "Scale factor: ${scale}"
print "Final box size: ${boxsize}"

# set variable to be used as temperature in any future fixes
variable temperature equal 1.0

# define particle groups in case we need to do any analysis on them
group   center type 1
group   irr_binding type 2 3 4

# define pair potential
pair_style lj/cut ${rcut}
pair_coeff * * 1.0 ${sigma} ${rcut}           # Purely repulsive interactions
pair_modify shift yes  # shift the potential up so that V(rcut)=0

# define bond potential
bond_style      harmonic            # Harmonic bonds
bond_coeff      1 50.0 ${r0}        # Bond coefficients. Spring constant 50.0,
bond_coeff      2 50.0 ${r0}        # Bond coefficients. Spring constant 50.0,

# use a large-radius neighbor list during compression phase
comm_modify mode single cutoff 9.0
neighbor 7.0 bin
neigh_modify delay 20 every 2 check yes

# initialize velocities using Maxwell-Boltzmann distribution
velocity all create ${temperature} ${seed}

# minimize potential energy to avoid large forces in first timestep
minimize 1e-6 1e-10 10000 100000
reset_timestep 0

fix	step all nve            # Time integrator

variable sleft equal "cpuremain"         # Est. simulation time remaining (seconds)
variable hleft equal "v_sleft / 3600.0"  # Est. simulation time remaining (hours)

# Custom output to log file and screen
thermo_style custom v_hleft time step pe ke etotal epair emol press
thermo 5000

# We use the same seed for the thermostat as for the initialization.
fix    temp all langevin ${temperature} ${temperature} 300 ${seed}

# Compress the simulation box to the desired density
fix    compress all deform 1 x scale ${scale} y scale ${scale} z scale ${scale}
run ${compression_steps}
unfix compress

# Save simulation box to a data file (positions, bonds, etc)
# This is used by analysis/plotting routines to have the molecular topology at the beginning
write_data monomer_equilibration_topology_${n_mon}_conc${conc}_seed${seed}.dat pair ij

# This block sets up a mean square displacement calculation. Can be freely removed
compute cmd center msd
variable time_now equal time
variable t0  equal ${time_now}
variable t   equal "time - v_t0"
variable dx2 equal c_cmd[4]
fix print all print 5000 "$t ${dx2}" file msd_${fileoutput}.txt screen no

# Change the neighbor list settings to better match the next bit (no more box volume changes, higher density)
comm_modify mode single cutoff 2.0
neighbor 0.5 bin
neigh_modify delay 4 every 2 check yes

# Setup up dump file for positions of particles as a function of time and run equilibration
dump traj all custom 500 monomer_equilibration_trajectory${n_mon}_4_arms_conc${conc}_seed${seed}.bin id mol type x y z
run 20000
undump traj

# syntax of the bond/create fix: fix ID group-ID bond/create Nevery itype jtype Rmin bondtype keyword
# so we are checking every timestep if a particle of type 2 and one of type 3 are within r_bond from
# each other and if so we create a bond of type 2 between them and turn both of them into type 4.
variable r_bond equal "v_sigma"
fix newbond all bond/create 1 2 3  ${r_bond} 2 iparam 1 4 jparam  1 4
thermo_style custom v_hleft time step pe ke etotal epair emol press f_newbond[2]

# Save simulation box to another data file
write_data gelation_initial_topology_${n_mon}_conc${conc}_seed${seed}.dat pair ij

# Set up regular dump file for positions
dump traj all custom 500 gelation_trajectory${n_mon}_4_arms_conc${conc}_seed${seed}.bin id mol type x y z

# Set up special dump file for bond information (used by e.g. Ovito to know which particles are bonded)
variable type4 atom "type == 4"
group newlinks dynamic all var type4 every 1000
compute bonds all property/local batom1 batom2 btype
dump crosslinks all local 500 gelation_bondinfo_${fileoutput}.dump c_bonds[1] c_bonds[2] c_bonds[3]

# count how many new crosslinks are created and save to file
variable countbond4 equal f_newbond[2]
variable step0 equal step

fix print_bondsformed all print 5000 "${step0} ${countbond4}" file crosslink_${fileoutput}.txt screen no

run 200000

# Finally store the current snapshot to apply percolation checks and stuff.
# write_restart final_snapshot_${fileoutput}.restart
write_data final_snapshot_${fileoutput}.data pair ij
""")

# Running Gelation Simulations
This section contains preparation code block and a simulation code block.

In [ ]:
# Set the parameters of the next simulations

lammps_dict = create_lammps_dict(concentration=8, Nstars=8, seed=188)  # should percolate in y and z
# lammps_dict = create_lammps_dict(concentration=8, Nstars=8, seed=189)  # should percolate in all directions
print(lammps_dict)

# Create the output folder
# Note that the simulation does not write directly to Google Drive but stores data
# In the Runtime first and moves it after the simulation is finished.
output_folder, output_fullpath = create_output_folder(lammps_dict["fileoutput"])

In [ ]:
# RUN SIMULATION (should take about 30 seconds with the default script)

# Make the output directory the working directory and start LAMMPS
os.chdir(output_fullpath)
lmp = lammps.lammps()

# Set variables one would normally set via commandline options
for var, value in lammps_dict.items():
    lmp.command(f'variable {var} string "{value}"')

# Execute the LAMMPS script
lmp.file(os.path.join("..", config["scripts_folder"], config["script_filename"]))

# Save final state in some Python variables
final_box = lmp.extract_box()
box_dimensions = np.asarray(final_box[1]) - np.asarray(final_box[0])
final_pos = np.reshape(lmp.gather_atoms("x", 1, 3), (-1, 3))
final_bonds = np.reshape(lmp.gather_bonds()[1],(-1, 3))

# Close the LAMMPS process (flushes buffered outputs)
lmp.close()

# Download results
os.chdir("..")
zip_results(output_folder)

# Percolation analysis
This uses the variables that were extracted from the lammps object at the end of the last simulation run to see whether the gelation process led to a percolating gel.

The `perconet` package employed was written by Chiara Raffaelli and Wouter Ellenbroek. For documentation see [https://perconet.readthedocs.io/](https://perconet.readthedocs.io/)

In [ ]:
# Create PeriodicNetwork object for percolation analysis
topology = perconet.PeriodicNetwork(n=len(final_pos), verbose=False)

# Add bonds (including boundary-crossing info) to PeriodicNetwork object
for bond in final_bonds:
    i = bond[1] - 1  # LAMMPS is 1-based but perconet is 0-based
    j = bond[2] - 1
    bondvec = final_pos[j] - final_pos[i]
    boundary_crossing_vector = np.round(bondvec/box_dimensions)
    topology.add_edge(i, j, boundary_crossing_vector)

# Find boundary-crossing loops in network
loopfinder = perconet.LoopFinder(topology, verbose=False)
loops, Nloops = loopfinder.get_independent_loops()

if Nloops == 0:
    print("Network does not percolate")
else:
    print(f"Network percolates in {Nloops} independent directions")
    for loop in loops:
        if np.all(loop == [1, 0, 0]):
            print("  Network percolates in x-direction")
        if np.all(loop == [0, 1, 0]):
            print("  Network percolates in y-direction")
        if np.all(loop == [0, 0, 1]):
            print("  Network percolates in z-direction")
    print("List of independent loops:")
    for loop in loops:
        print(" *", loop)